## Árvore B

O código abaixo ilustra a inclusão, busca, exclusão e percursos em árvores B, com grau máximo 2*t= 4, (t=2).

Árvores B são estruturas usadas para criação de índices em memória secundária. O tamanho dos nós é geralmente o tamanho de uma página na memória secundária, ou múltiplos do tamanho da página, com o objetivo de minimizar o número de operações de leitura em disco.

Para permitir testes online das operações de inclusão/exclusão, esta versão não possui as operações de acesso ao disco.

In [1]:
#include <iostream>      //Biblioteca para operações de entrada e saída
#include <stdlib.h>      //Biblioteca para manipulação de memória e operações de conversão de tipos
#include <chrono>        //Biblioteca para análise de tempo de execução de códigos e métodos
#include <random>        //Biblioteca para geração de elementos aleatórios
#include <vector>        //Biblioteca para criação de vetores
#include <numeric>       //Biblioteca para uso de funções matemáticas
#include <cmath>         //Biblioteca para uso de funções matemáticas

using namespace std;

#define GRAU_MAXIMO 200  //Número máximo de filhos (Max 2t = 4) -> grau 2
#define GRAU_MINIMO 100  //Número mínimo de filhos (Min t = 2) -> grau 2

#### Estrutura para uma árvore B simples
A árvore tem um conjunto de chaves e de nós filhos. O número de nós filhos é sempre 1 unidade maior que o número de chaves.

Método para atribuição de grau mínimo e máximo da árvore B

In [2]:
class tNo {                     //Criação do agregado
public:
  int chave [GRAU_MAXIMO-1];    //Número de chaves criado com o tamanho máximo (2t-1)
  tNo *p [GRAU_MAXIMO];         //Nó interno sempre vai ter n+1 filhos (Ponteiro para os filhos)
  int tam;                      //Número de chaves atualmente no nó
}

#### Inicialização do nó
Função para inicialização do nó, com alocação de memória e atribuição de valores NULL para os ponteiros e 0 para o tamanho.

In [3]:
tNo *criaNo (){                              //Criação do método
    tNo *n = (tNo *)malloc (sizeof (tNo));   //Aloca a memória para a criação de um nó
    n->tam = 0;                              //Define que o nó recém criado tem 0 chaves (tamanho 0)
    for (int i=0; i < GRAU_MAXIMO ;i++)      //Define cada posição dentro do nó como null, para que não exista nenhum registro de memória ali presente
       n->p[i] = NULL;
    return n;
}

Função para imprimir a árvore B em ordem

In [4]:
void printBTree(tNo *node) {
  if (node) {                                //Verifica se existe um nó
    for (int i = 0; i < node->tam; i++) {    //Vai lendo e imprimindo todas as chaves do nó atual
      cout << node->chave[i] << " ";
    }
    cout << endl;                            //Adicione essa linha para separar as chaves de cada nó
    for (int i = 0; i <= node->tam; i++) {   //Filhos do nó atual
      printBTree(node->p[i]);
    }
  }
}


#### Conversão da string em um vetor de elementos inteiros
Função que lê um token, separado por espaço, e converte para um número inteiro.

In [5]:
int token_to_num(const char *str, int *indice){           //Str é o número a ser convertido, índice é a posição atual
    char token[100];                                      //Array que armazena o token de tamanho 100 antes de converter
    int i = 0;                                            //Contador i
    while (str[*indice] != '\0' && str[*indice] != ' '){  //Laço de repetição que será executado até encontrar um espaço nulo (\0) ou um espaço em branco
        token[i] = str[*indice];                          //Copia caracteres individualmente para o token
        i++;                                              //Soma +1 no contador
        (*indice)++;                                      //Soma +1 no ponteiro do índice
    }
    token[i] = '\0';                                      //Caracter nulo colocado no fim do array
    (*indice)++;                                          //Soma +1 no ponteiro do índice por conta desse caractere nulo adicionado
    return atoi(token);                                   //Faz a conversão
}

#### Altura da árvore
Calcula a altura da árvore. Como a árvore B é sempre balanceada, a contagem da altura é feita descendo pelo primeiro ponteiro à esquerda até o nó folha.

In [6]:
int altura (tNo *p) {                     //Criação do método
    int i = 0;                            //Contador i
    while (p->p[0] != NULL){              //Enquanto o primeiro filho não for nulo
        p = p->p[0];                      //Vai para o próximo filho
        i ++;                             //Soma +1
    }
    return i;
}

#### Divisão (Split) do nó interno - Operação de inclusão
Função que divide um nó de uma árvore B, usada na inclusão, quando o nó está cheio. Um nó está cheio quando o número de chaves é igual ao grau máximo - 1.

In [7]:
void splitNo (tNo *pai, int indice, tNo *no) {           //Criação do método. O índice pode ser entendido como a ordenação dos filhos do nó pai (1º, 2º, .. filho)
    tNo *novo = criaNo();                                //Cria um novo nó
    novo->tam = GRAU_MINIMO - 1;                         //Tamanho do nó novo é o tamanho mínimo de um nó (t-1)

    for (int i=0; i < GRAU_MINIMO-1; i++){               //Laço de repetição para atribuição das chaves da segunda metade do nó corrente para o novo nó (Acima da mediana)
        novo->chave[i] = no->chave[i+GRAU_MINIMO];
    }
                                                         //Dado que o nó não é uma folha e possui filhos
    if (no->p[0]  != NULL) {                             //novo->chave[i]
        for (int i=0; i <= GRAU_MINIMO-1; i++)           //Atribuição dos pointeiros da segunda metade do nó corrente
            novo->p[i] = no->p[i + GRAU_MINIMO];
    }
    no->tam = GRAU_MINIMO - 1;                           //Redefine o tamanho do nó original para o tamanho mínimo de um nó

    for (int i = pai->tam;  i > indice; i--)             //Laço de repetição que desloca os ponteiros do nó pai para a direita
        pai->p[i+1] = pai->p[i];

    pai->p[indice+1] = novo;                             //Último indice do nó pai aponta para o novo nó

    for (int i = pai->tam-1; i >= indice; i--)           //Laço de repetição que desloca as chaves do nó pai para a direita
        pai->chave[i+1] = pai->chave[i];

    pai->chave[indice] = no->chave[GRAU_MINIMO-1];       //Realiza o movimento de subida da chave (mediana)
    pai->tam = pai->tam +1;                              //Aumenta o tamanho do nó pai para que seja possível caber a chave (mediana)
}

#### Divisão (Split) do nó raíz - Operação de inserção
Função que faz o split da árvore B na raiz. É diferente do split dos demais nós, pois são criados 2 novos nós: a nova raiz e o novo nó à direita.

In [8]:
tNo *splitRaiz(tNo * raiz){                            //Criação do método
    tNo *novaRaiz = criaNo();                          //Cria uma nova raiz (criação de um novo nó)
    splitNo(novaRaiz, 0, raiz);                        //Divisão da raiz, onde o nó pai é a novaRaiz, índice 0 e nó corrente igual a raiz atual
    novaRaiz->p[0]=raiz;                               //Atribuição da nova raiz
    novaRaiz->p[0]->tam = GRAU_MINIMO - 1;             //Define o tamanho da nova raiz como o tamanho mínimo de um nó
    return novaRaiz;                                   //Retorna a nova raíz
}

#### Operação de inserção

Funcão para inclusão de nova chave na árvore B, dado um nó raiz. O split é feito na descida, isto é, sempre que encontrar um nó cheio, irá chamar a função __splitNo__, e continuará a descida.

In [9]:
tNo* inclui (tNo *no, int chave, tNo* raiz){             //Criação do método. Nó refere-se ao nó que está sendo analisado, a chave que se deseja inserir e a raiz da árvore
    int i = no->tam - 1;                                 //Índice final - última posição dentro do nó (Ex: tamanho 10, índice vai de 0 a 9, com i = 9)
    tNo *filho;                                          //Nó filho do nó corrente onde ocorrerá a inserção

    if (no->tam == GRAU_MAXIMO - 1 && no == raiz) {      //Tratamento específico para a raiz - Caso o nó corrente seja a raiz da árvore B, fazemos a divisão específica do nó raiz
        raiz = splitRaiz(raiz);                          //Divisão da raiz
        no = raiz;                                       //Nó corrente analisado passa a ser a raiz
        i = no->tam - 1;                                 //Índice final atualizado para ser a última posição da raiz (Ex: tamanho 10, índice vai de 0 a 9, com i = 9)
    }

    if (no->p[0] == NULL){                              //Tratamento para o caso de inclusão no nó folha
        while (i >= 0 && chave < no->chave[i]) {        //Laço de repetição que será executado enquanto existir elementos no nó corrente e enquanto a chave que se deseja inserir for menor que as chaves já existentes no nó
            no->chave[i+1] = no->chave[i];              //Movimento da chave existente que é maior que a chave que se deseja inserir à direita em uma posição ("Abrindo espaço")
            i--;                                        //Decrementa o índice de última posição para avaliar a próxima chave existente
        }
        no->chave[i+1] = chave;                         //Caso não exista elementos no nó analisado ou a chave que se deseja inserir seja maior que a chave existente, inserimos no espaço criado
        no->tam = no->tam+1;                            //Incremento o tamanho do nó em 1 por conta da chave inserida
    }
    else {                                              //Caso seja apenas um nó interno
        while (i >= 0 && chave < no->chave[i])          //Laço de repetição que será executado enquanto existir elementos no nó corrente e enquanto a chave que se deseja inserir for menor que as chaves já existentes no nó
            i--;                                        //Decrementa o índice de última posição para avaliar a próxima chave existente
        i++;                                            //Incrementa o índice de última posição para avaliar a próxima chave existente
        filho = no->p[i];                               //Nó filho da primeira chave existente no nó que é maior que a chave que desejamos inserir
        if (filho->tam == GRAU_MAXIMO - 1){             //Se o filho tiver o tamanho máximo permitido, será necessário fazer o split (Desce na árvore e faz o split se necessário)
            splitNo(no, i, filho);                      //Divisão do nó filho
            if (chave > no->chave[i])                   //Ele vai avaliar novamente o nó corrente após a subida da chave (mediana) 
                i ++;                                   //Incrementa o índice da última posição para avaliar a próxima chave existente  
        }
        inclui(no->p[i], chave, raiz);                  //Recursividade aplicada para processo de subida, casos os nós apresentem o limite máximo de chaves - Será executava no máximo até a raiz
    }
    return raiz;                                        //Retorna a raíz
}

#### Operação de busca
Busca em árvore B. Retorna o nó com a chave encontrada.

In [10]:
tNo *busca (tNo *no, int chave) {                      //Criação do método. Nó refere-se ao nó que está sendo analisado e a chave, refere-se a chave buscada
    if( no != NULL ){                                  //Condição de nó não nulo
        for(int i=0; i < no->tam; i++) {               //Laço de repetição que percorre todas as chaves do nó analisado
            if( no->chave[i] == chave )                //Condição da chave avaliada ser igual a chave buscada
                return no;                             ///Retorna o nó com a chave encontrada
            else                                       //Caso a chave avaliada seja diferente da chave buscada
                if( chave < no->chave[i] )             //Caso a chave buscada seja menor
                    return busca( no->p [i], chave );  //Recursividade da busca aplicada ao filho à esquerda da chave
        }
        return busca( no->p[ no->tam ], chave );       //Recursividade da busca aplicada ao filho da última posição do nó (à direita da última chave)
    }
    else
        return NULL;
}

#### Funções auxiliares para operação de remoção

Função auxiliar para retornar o índice da chave em um nó.

In [11]:
int indiceChave(tNo *no, int chave){
    int i;
    for (i = 0; i < no->tam && no->chave[i] != chave; i++);
    return i;
}

Função auxiliar para retornar o índice da sub-árvore onde se encontra a chave, dado um nó qualquer

In [12]:
int posicaoSubArvore(tNo *no, int chave){
    int i = no->tam - 1;
    while (i >= 0 && chave < no->chave[i])
        i--;
    i++;
    return i;
}

Função auxiliar para deslocar todas as chaves e os ponteiros de um nó uma posição para a direita, até um índice determinado (Ex: [0,1,2] vira [1,2,3], [0+1, 1+1, 2+1])

In [13]:
void deslocaDireita(tNo *no){
    int i;
    for (i=no->tam ; i > 0 ; i--){
        no->chave[i] = no->chave[i-1];
        no->p[i+1] = no->p[i];
    }
    no->p[i+1] = no->p[i];
}

Função auxiliar para deslocar as chaves e os ponteiros de um nó uma posição para a esquerda, até um índice determinado. (Ex: [1,2,3] vira [0,1,2], [1-1, 2-1, 3-1])

In [14]:
void deslocaEsquerda(tNo *no, int indice){
    int i;
    for (i=indice ; i < no->tam ; i++){
        no->chave[i] = no->chave[i+1];
        no->p[i] = no->p[i+1];
    }
    no->p[i] = no->p[i+1];
}

Função auxiliar para fusionar (unir) dois nós passados como parâmetro, e retornar o nó fusionado (unido).

Fusiona o nó direito no nó esquerdo, ajusta as chaves e os ponteiros do nó, e libera o nó direito da memória.

In [15]:
tNo* fusionaNo(tNo *esq, int chave, tNo * dir){                     //Criação do método
    int tamEsq = esq->tam;                                          //Variável que contém o tamanho do nó esquerdo
    esq->chave[tamEsq] = chave;                                     //Última chave do filho esquerdo recebe a chave do nó "a ser descida"
    int i;                                                          //Contador i
    tamEsq++;                                                       //Incrementa o tamanho do nó esquerdo
    esq->tam ++;                                                    //Incrementa o tamanho do nó esquerdo
    for (i=0; i < dir->tam; i++){                                   //Laço de repetição que copia chaves e ponteiros da direita para no esquerdo
        esq->chave[tamEsq + i] = dir->chave[i];
        esq->p[tamEsq + i] = dir->p[i];
    }
    esq->p[tamEsq + i] = dir->p[i];                                 //Fusão do filho direito com o esquerdo
    esq->tam = esq->tam + dir->tam;                                 //Tamanho total (soma dos tamanhos dos filhos direito e esquerdo)
    free(dir);                                                      //Liberação do nó direito na memória
    return esq;                                                     //Retorna nó fusionado
}

#### Operação de remoção

Processo de exclusão em árvore B. Possui 3 casos:
1. Chave está no nó e é folha
2. Chave está no nó e é interno
    - se filho esquerdo tem ao menos t chaves
    - se filho direito tem ao menos t chaves
    - se filhos direito ou esquerdo não tenham ao menos t chaves
3. Chave não está no nó interno
    - se a raiz da subárvore tem t-1 chaves mas irmão imediato (esquerdo ou direito) tem ao menos t chaves
    - se irmãos imediatos possuem t-1 chaves


In [16]:
tNo* exclui (tNo *no, int chave, tNo* raiz){                                          //Criação do método. Nó refere-se ao nó analisado, chave é a chave k que se pretende remover e a raiz da árvore B
    int j, i = no->tam - 1;                                                           //Índices finais i e j - última posição dentro do nó (Ex: tamanho 10, índice vai de 0 a 9, com i, j = 9)
    int aux=0, excluiuCaso2 = 0;                                                      //Variavel auxiliar iniciada em 0, variavel auxiliar para o segundo caso
    tNo *filho, *irmao, *sucessor, *antecessor;                                       //Ponteiros para o filho, irmão, sucessor e antecessor do nó avaliado

    if (indiceChave(no, chave) == 0 && no->tam == 1 && no == raiz) {                  //Condição do nó ser raíz de tamanho 1 com índice de primeiro valor igual a 0
        //cout << "\nExcluiu nó raiz";                                                //Informar sobre a exclusão do nó raiz e desaloca recurso (libera memória)
        free(no);                                                                     //Desaloca recursos dinâmicos (liberação de memória)
        return NULL;
    }
    if (no->p[0] == NULL){                                                            //Caso 1: A chave está no nó e o nó é folha
        //cout << "\nCaso 1: nó folha";                                               //Informa em qual caso estamos (Caso 1)
        j = indiceChave(no, chave);                                                   //Índice da chave que se quer remover
        if (j == no->tam) return NULL;                                                //Condição do índice ser o tamanho do nó, ou seja, não encontrou a chave no método indiceChave
        while (j < no->tam) {                                                         //Laço de repetição que percorre todas as chaves do nó
            no->chave[j] = no->chave[j+1];                                            //Desloca as chaves de um nó uma posição para a esquerda, ou seja, ele para de apontar para determinada chave
            j++;                                                                      //Incrementa o índice
        }
        no->tam = no->tam-1;                                                          //Decrementa o tamanho do nó por conta da exclusão
    }
    else {                                                                            //Caso não esteja em um nó folha, ou seja, esteja em um nó intermediário, desce na árvore B e faz o merge se necessário
        j = indiceChave(no, chave);                                                   //j é o índice da chave no nó
        i = posicaoSubArvore(no, chave);                                              //i é o índice da sub-árvore onde se encontra a chave, dado um nó qualquer

        if (j < no->tam) {                                                            //Caso 2: encontrou a chave em nó interno
            if (no->p[j] != NULL && no->p[j]->tam >= GRAU_MINIMO) {                   //Condição do nó ter filho à esquerda da chave avaliada e esse filho estar com grau mínimo (t chaves)
                //cout << "\nCaso 2: antecessor";                                     //Informa em qual caso estamos (Caso 2: antecessor)
                antecessor = no->p[j];                                                //Antecessor recebe o filho à esquerda de um nó
                while (antecessor->p[antecessor->tam - 1] != NULL)                    //Laço de repetição executado enquanto o .......
                    antecessor = antecessor->p[antecessor->tam - 1];                  //Antecessor se torna o filho à esquerda da última chave do nó
                no->chave[j] = antecessor->chave[antecessor->tam -1];
                exclui(no->p[j], antecessor->chave[antecessor->tam-1],raiz);          //Decrementa o tamanho do nó por conta da exclusão
            } else {
                if (no->p[j+1] != NULL && no->p[j+1]->tam >= GRAU_MINIMO) {
                    //cout << "\nCaso 2: sucessor";                                   //Informa em qual caso estamos (Caso 2: sucessor)
                    sucessor = no->p[j+1];
                    while (sucessor->p[0] != NULL)
                        sucessor = sucessor->p[0];
                    no->chave[j] = sucessor->chave[0];
                    exclui(sucessor, sucessor->chave[0], raiz);
                    }
                else {
                    //cout << "\nCaso 2: sem grau mínimo";                           //Informa em qual caso estamos (Caso 2: sem grau mínimo)
                    filho = fusionaNo(no->p[j], no->chave[j], no->p[j+1]);
                    no->chave[j] = no->chave[j+1];
                    deslocaEsquerda(no, j+1);
                    no->tam --;
                    exclui(filho, chave, raiz);
                }
            }
        }
        else {                                                                      //Caso 3: não encontrou a chave em nó interno
            filho = no->p[i];                                                       //Sub arvore da chave
            if (no->p[i]->tam == GRAU_MINIMO - 1){
                if ( (i-1)>=0 && no->p[i-1]->tam >= GRAU_MINIMO){
                    //cout << "\nCaso 3: a.esq)";                                   //Informa em qual caso estamos (Caso 3: filho esquerdo com ao menos t chaves)
                    irmao = no->p[i-1];
                    aux = no->chave[i-1];
                    irmao->tam --;
                    no->chave[i-1] = irmao->chave[irmao->tam];
                    deslocaDireita(filho);
                    filho->tam ++;
                    filho->chave[0] = aux;
                    filho->p[0] = irmao->p[irmao->tam+1];
                }
                else {
                    if ((i+1) <= no->tam && no->p[i+1]->tam >= GRAU_MINIMO){
                        //cout << "\nCaso 3: a.dir)";                              //Informa em qual caso estamos (Caso 3: filho direito com ao menos t chaves)
                        irmao = no->p[i+1];
                        aux = no->chave[i];
                        irmao->tam --;
                        no->chave[i] = irmao->chave[0];
                        deslocaEsquerda(irmao, 0);
                        filho->tam ++;
                        filho->chave[filho->tam-1] = aux;
                        filho->p[filho->tam] = irmao->p[0];
                    }
                    else {                                                          //Senão, realiza a fusao de nós
                        //cout << "\nCaso 3: b)";                                   //Informa em qual caso estamos (Caso 3: irmãos imediatos possuem t-1 chaves)
                        if (i == 0){                                                //Primeiro nó fusiona com o direito
                            filho = fusionaNo(filho, no->chave[0], no->p[i+1]);
                            no->chave[0] = no->chave[1];
                            deslocaEsquerda(no, i+1);
                        } else {                                                   //Fusiona agora com o esquerdo
                            filho = fusionaNo(no->p[i-1], no->chave[i-1], filho);
                            no->chave[i-1] = no->chave[i];
                            deslocaEsquerda(no, i);
                            i--;
                        }
                        no->tam --;                                                //Decrementa o tamanho do nó por conta da exclusão
                    }
                }

            }
        exclui( no->p[i], chave, raiz);
        }
    }
    return raiz;
}


#### Montagem de uma árvore B
Função que lê a string de entrada com as chaves, chama a função de inclusão e retorna a raiz da árvore.

In [17]:
tNo* montaarvore(const int numeros_usados[], int tamanho){
    tNo *raiz = criaNo();
    for (int i = 0; i < tamanho; i++) {
        raiz = inclui(raiz, numeros_usados[i], raiz);
    }

    return raiz;
}

#### Função Main

Função para inicialização do programa

In [18]:
void iniciaprograma(){

    std::vector<double> tempos;                                                                        //Vetor que vai armazenar os tempos de execução
    int repeticoes = 100;                                                                              //Número de vezes que o código será executado

    for(int rep = 0; rep < repeticoes; rep++){
        tNo *raiz = NULL;

        std::mt19937 rng(5);                                                                          //Uso do método Mersenne Twister para geração de números aleatórios. Semente 5 para garantir a reprodutibilidade do experimento
        std::uniform_int_distribution<int> dist(1, 10000);                                            //Distribuição uniforme, onde todos os valores tem iguais chances de serem escolhidos. Mais números disponíveis que escolhidos
        int tamanho = 5000;
        int numeros_usados[tamanho];                                                                  //Criação de um array com o tamanho do teste desejado
        for (int i = 0; i < tamanho; i++) {                                                           //Array preenchido
            numeros_usados[i] = dist(rng);
        }

        raiz=montaarvore(numeros_usados, tamanho);                                                    //Montagem da árvore B
            //for (int i = 0; i < tamanho; i++) {                                                     //Imprimindo os valores
                //std::cout << "Número " << i << ": " << numeros_usados[i] << std::endl;
            //}
            //std::cout << "Árvore B: ";                                                              //Impressão da árvore B
            //printBTree(raiz);
            //std::cout << std::endl;

        auto start = std::chrono::high_resolution_clock::now();                                       //Inicia contagem do tempo de execução após montagem da árvore

        //tNo* resultado = busca(raiz, 2220);                                                         //Valores de Teste: Valor 0
        //tNo* resultado = inclui(raiz,10001,raiz);                                                   //Valores de Teste: Valor fora do range (1,10000) para ter certeza que não está sendo usado
        tNo* resultado = exclui(raiz,2220,raiz);                                                      //Valores de Teste: Valor 2220 (1º valor gerado para a semente 5) - Garantindo existência / presença do valor na árvore

        auto stop = std::chrono::high_resolution_clock::now();                                        //Finalização da mediçãõ do tempo de execução
        auto duration = std::chrono::duration_cast<std::chrono::microseconds>(stop - start);          //Duração do tempo de execução (fim - inicio)
        tempos.push_back(duration.count());                                                           //Adiciona o tempo encontrado na execução no vetor
    }

    double media = std::accumulate(tempos.begin(), tempos.end(), 0.0) / repeticoes;                   //Somar todos os valores entre o inicio e o fim do vetor de tempos, com valor inicial para operação em 0.0
    double soma_quadrados_diferencas = 0.0;                                                           //Variável auxiliar iniciada em 0 que irá conter a informação da soma dos quadrados da diferença entre o tempo de execução e a média

    for (double tempo : tempos) {                                                                     //Laço de repetição que irá ser útil no cálculo do desvio padrão do vetor de tempos para criação das variáveis diferena entre os tempos e a soma dos quadrados
        double diferenca = tempo - media;
        soma_quadrados_diferencas += diferenca * diferenca;
    }

    double variancia = soma_quadrados_diferencas / (repeticoes - 1);                                 //Variável para guardar a variância usada no cálculo do desvio padrão
    double desvio_padrao = std::sqrt(variancia);                                                     //Desvio padrão por definição é a raíz quadrada da variância


    std::cout << "Média do tempo de execução: " << media << " microsegundos" << std::endl;           //Print da média dos tempos de execução
    std::cout << "Desvio padrão: " << desvio_padrao << " microsegundos" << std::endl;                //Print do desvio padrão dos tempos de execução

    std::cout << "Tempos de execução: ";                                                             //Print dos tempos de execução
    for (double tempo : tempos) {
        std::cout << tempo << " ";
    }
    std::cout << std::endl;

}

#### Execução da main

In [19]:
iniciaprograma();

Média do tempo de execução: 15.85 microsegundos
Desvio padrão: 4.93058 microsegundos
Tempos de execução: 46 14 15 14 21 15 20 13 14 14 15 12 13 28 24 19 21 21 21 14 14 15 16 15 14 14 16 14 15 15 15 13 15 15 15 14 14 15 16 13 15 14 15 14 14 16 16 14 14 15 14 15 15 15 15 15 46 13 16 15 15 15 17 21 15 15 15 14 16 15 15 14 14 14 14 15 14 16 15 14 14 14 15 14 15 15 16 14 14 14 14 14 16 15 16 13 13 14 15 14 
